In [1]:
from __future__ import print_function, division, absolute_import

## Load the DeepSEA Beluga model

In [2]:
import torch
from torch.utils.serialization import load_lua

#The model is downloaded from
# wget http://deepsea.princeton.edu/media/code/expecto/resources.tar.gz
pytorch_model = load_lua('/home/ashrikumar/scratch/ashrikumar/models/ExPecto/resources/deepsea.beluga.2002.cpu')

In [3]:
pytorch_model

nn.Sequential {
  [input -> (0) -> (1) -> (2) -> output]
  (0): nn.Sequential {
    [input -> (0) -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> output]
    (0): nn.SpatialConvolution(4 -> 320, 1x8)
    (1): nn.ReLU
    (2): nn.SpatialConvolution(320 -> 320, 1x8)
    (3): nn.ReLU
    (4): nn.Dropout(0.2000)
    (5): nn.SpatialMaxPooling(1x4, 1, 4)
    (6): nn.SpatialConvolution(320 -> 480, 1x8)
    (7): nn.ReLU
    (8): nn.SpatialConvolution(480 -> 480, 1x8)
    (9): nn.ReLU
    (10): nn.Dropout(0.2000)
    (11): nn.SpatialMaxPooling(1x4, 1, 4)
    (12): nn.SpatialConvolution(480 -> 640, 1x8)
    (13): nn.ReLU
    (14): nn.SpatialConvolution(640 -> 640, 1x8)
    (15): nn.ReLU
  }
  (1): nn.Sequential {
    [input -> (0) -> (1) -> (2) -> (3) -> (4) -> output]
    (0): nn.Dropout(0.5000)
    (1): nn.Reshape(67840)
    (2): nn.Linear(67840 -> 2003)
    (3): nn.ReLU
    (4): nn.Linear(2003 -> 2002)
  }
  (2): nn.Sigmoid
}

## Create a keras model with the right architecture

In [4]:
import keras
import numpy as np

keras_model = keras.models.Sequential()
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=320, kernel_size=8,
            strides=1, padding="valid",
            batch_input_shape=(None, 2000, 4)))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=320, kernel_size=8,
            strides=1, padding="valid",
            batch_input_shape=(None, 1000, 4)))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.2))
keras_model.add(keras.layers.pooling.MaxPooling1D(
           pool_size=4, strides=4, padding="valid"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=480, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=480, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.2))
keras_model.add(keras.layers.pooling.MaxPooling1D(
           pool_size=4, strides=4, padding="valid"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=640, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.convolutional.Conv1D(
            filters=640, kernel_size=8,
            strides=1, padding="valid"))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dropout(0.5))
keras_model.add(keras.layers.core.Permute((2,1)))
keras_model.add(keras.layers.core.Flatten())
keras_model.add(keras.layers.core.Dense(2003))
keras_model.add(keras.layers.Activation("relu"))
keras_model.add(keras.layers.core.Dense(2002))
keras_model.add(keras.layers.Activation("sigmoid"))
keras_model.build()

/home/ashrikumar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Port the DeepSEA weights over

In [5]:
import numpy as np

conv1_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[0].weight
                           .detach().cpu().numpy())[:,[0,2,1,3]]
                           .transpose((2,1,0)))
conv1_bias = (pytorch_model.modules[0].modules[0].bias
                           .detach().cpu().numpy())
keras_model.layers[0].set_weights([conv1_weight, conv1_bias])


conv2_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[2].weight
                           .detach().cpu().numpy())
                           .transpose((2,1,0)))
conv2_bias = (pytorch_model.modules[0].modules[2].bias
                           .detach().cpu().numpy())
keras_model.layers[2].set_weights([conv2_weight, conv2_bias])


conv3_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[6].weight
                           .detach().cpu().numpy())
                           .transpose((2,1,0)))
conv3_bias = (pytorch_model.modules[0].modules[6].bias
                           .detach().cpu().numpy())
keras_model.layers[6].set_weights([conv3_weight, conv3_bias])


conv4_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[8].weight
                           .detach().cpu().numpy())
                           .transpose((2,1,0)))
conv4_bias = (pytorch_model.modules[0].modules[8].bias
                           .detach().cpu().numpy())
keras_model.layers[8].set_weights([conv4_weight, conv4_bias])


conv5_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[12].weight
                           .detach().cpu().numpy())
                           .transpose((2,1,0)))
conv5_bias = (pytorch_model.modules[0].modules[12].bias
                           .detach().cpu().numpy())
keras_model.layers[12].set_weights([conv5_weight, conv5_bias])


conv6_weight = (np.squeeze(pytorch_model.modules[0]
                           .modules[14].weight
                           .detach().cpu().numpy())
                           .transpose((2,1,0)))
conv6_bias = (pytorch_model.modules[0].modules[14].bias
                           .detach().cpu().numpy())
keras_model.layers[14].set_weights([conv6_weight, conv6_bias])


dense1_weight = (np.squeeze(pytorch_model.modules[1].modules[2].weight
                            .detach().cpu().numpy())).transpose((1,0))    
dense1_bias = (pytorch_model.modules[1].modules[2].bias
                            .detach().cpu().numpy())
keras_model.layers[19].set_weights([dense1_weight, dense1_bias])


dense2_weight = (np.squeeze(pytorch_model.modules[1].modules[4].weight
                            .detach().cpu().numpy())).transpose((1,0))   
dense2_bias = (pytorch_model.modules[1].modules[4].bias
                            .detach().cpu().numpy())
keras_model.layers[21].set_weights([dense2_weight, dense2_bias])

In [6]:
keras_model.save("deepseabeluga_keras.h5")

## Make sure the conversion is correct

In [1]:
import numpy as np
import torch
from torch.utils.serialization import load_lua

pytorch_model = load_lua('/home/ashrikumar/scratch/ashrikumar/models/ExPecto/resources/deepsea.beluga.2002.cpu')

np.random.seed(1)
dummy_data = np.random.random((10,4,2000,1))
#one-hot encode it
dummy_data = (dummy_data==np.max(dummy_data,axis=1)[:,None,:,:]).astype("float32")

pytorch_predictions = (pytorch_model.forward(torch.from_numpy(dummy_data))
                       .detach().cpu().numpy())

np.save("pytorch_preds.npy", pytorch_predictions)

Kernel may crash if both pytorch and keras are run in same session, so good to reload before running cell below

In [1]:
import numpy as np
from keras.models import load_model

np.random.seed(1)
dummy_data = np.random.random((10,4,2000,1))
#one-hot encode it
dummy_data = (dummy_data==np.max(dummy_data,axis=1)[:,None,:,:]).astype("float32")

keras_model = load_model("deepseabeluga_keras.h5")
pytorch_predictions = np.load("pytorch_preds.npy")
keras_predictions = keras_model.predict(dummy_data.squeeze().transpose(0,2,1)[:,:,[0,2,1,3]])
max_diff = np.max(np.abs(pytorch_predictions-keras_predictions))
print(max_diff)
assert max_diff < 10**-6

/home/ashrikumar/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ashrikumar/anaconda2/lib/python2.7/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


6.556511e-07
